# Mediapipe with YOLO

### Requirements

In [1]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2

import mediapipe as mp

import torch

c:\Users\spec3\anaconda3\envs\yoloV5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
# only interested in detecting person
yolo_model.classes = [0]

Using cache found in C:\Users\spec3/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-16 Python-3.8.13 torch-1.12.1+cpu CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 29.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_style = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

In [6]:
cap = cv2.VideoCapture("./video/[안방1열 직캠4K] 아이브 안유진 LOVE DIVE (IVE YUJIN FanCam)│@SBS Inkigayo_20220410.mp4")
while cap.isOpened():
    ret, frame = cap.read()
    h, w, d = frame.shape
    size = (w, h)
    
    if not ret: break
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    result = yolo_model(image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    MARGIN = 10
    
    for (xmin, ymin, xmax, ymax, confidence, clas) in result.xyxy[0].tolist():
        with mp_holistic.Holistic(min_detection_confidence=0.3, min_tracking_confidence=0.3) as holistic:
            results = holistic.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN])
            
            mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN, int(xmin)+MARGIN:int(xmax)+MARGIN],
                                      results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                      )
    cv2.imshow("T", image)
    if cv2.waitKey(10)&0xFF==ord("q"): break

cap.release()
cv2.destroyAllWindows()